
CSE 628 Term Project Report\
GPU and Multithreaded Comparision of Mean Face using img_align_celeba Dataset\
Zack Owens\
7/19/2023


In [ ]:
cd("..\\img_align_celeba")
pwd()

In [ ]:
using Base.Threads
num_threads = Threads.nthreads()

In [ ]:
# read in the CSV
using CSV
using DataFrames


df = CSV.read("./list_attr_celeba_2.csv",DataFrame)

In [ ]:
# split list into men list (hint use lpad)
males = filter(row->row.Male==1,df)
males.id

In [ ]:
# split list into women list (hint use lpad)
females = filter(row->row.Male==0,df)
females.id

## 1.2

Display the last 3 faces of men and the last 3 faces of women.

In [ ]:
#show last 3 mens and womens faces
using Images, FileIO, Colors

last_3_males = males.id[length(males.id)-2:length(males.id)]
last_3_females = females.id[length(females.id)-2:length(females.id)]

@time begin
    vector_N = Vector{Matrix{RGB{N0f8}}}([])
    for (i, f) in enumerate(last_3_males)
            push!(vector_N, load(f))
    end
end


@time begin
    vector_N_2 = Vector{Matrix{RGB{N0f8}}}([])
    for (i, f) in enumerate(last_3_females)
            push!(vector_N_2, load(f))
    end
end


mosaicview([vector_N ; vector_N_2]; 
fillvalue=0.5, npad=10, ncol=2)

In [ ]:
using Hwloc
using Base.Threads
topology()

In [ ]:
print(Sys.total_memory() / 2^20 /1000, " GB of RAM") 

In [ ]:
using Base.Threads

function mean_RGB(args...)
    
    c = RGB{Float64}(0,0,0)    
    w = 1.0 / length(args)
    for arg in args
       c += w * arg
    end
    
    return RGB{N0f8}(c)
end


In [ ]:
# read in the images for men at once
cd("..\\img_align_celeba")
pwd()


@time begin
    vector_N = Vector{Matrix{RGB{N0f8}}}([])
    for (i, f) in enumerate(males.id)
            push!(vector_N, load(f))
    end
end






In [ ]:

function calculate_batch_mean(images::Vector{Matrix{RGB{N0f8}}})    
    return mean_RGB.(images)
end

function batch_images(images::Vector{Matrix{RGB{N0f8}}}, batch_size::Int)
    num_images = length(images)
    num_batches = div(num_images, batch_size)
    
    batches = [images[(i - 1) * batch_size + 1:min(i * batch_size, num_images)] for i in 1:num_batches]
    return batches
end
function calculate_mean_parallel(images::Vector{Matrix{RGB{N0f8}}}, batch_size::Int)
    batches = batch_images(images, batch_size)
    num_batches = length(batches)
    
    mean_images = Vector{Matrix{RGB{N0f8}}}(undef, num_batches)
    
    @threads for i in 1:num_batches
        mean_images[i] = mean_RGB.(batches)
    end
    
    # Combine the mean images from different batches to get the final mean image
    mean_image = calculate_batch_mean(mean_images)
    
    return mean_image
end

In [ ]:

#working
#batch_images(vector_N[1:10],2)


#calculate_mean_parallel(vector_N[1:10],2)
#calculate_batch_mean(vector_N[1:10])

mean_RGB.(batch_images(vector_N[1:10],2)[1])

In [ ]:
#@time vector_mean = mean_RGB.(vector_N[1:100]...)

In [ ]:
#@time vector_mean = mean_RGB.(vector_N[1:500]...)

In [ ]:
size(vector_N[1])
vector_N[1][1]


In [ ]:
#@time vector_mean = mean_RGB_image(vector_N[1:100])

# read in the images for women at once
@time begin
    vector_N_2 = Vector{Matrix{RGB{N0f8}}}([])
    for (i, f) in enumerate(females.id)
            push!(vector_N_2, load(f))
    end
end
